# Movie Review Classification

Here we show how a computer can tell apart good and bad movie reviews. To do so, we create a "classifier": A device that learns data categorization patterns from a "labeled" data set.

First we get a lot of movie reviews and label them as good or bad. We will use these real reviews to train the classifier.

In [ ]:
# SETUP MAGIC
f = open("train.tsv")
cur = -1
all_X = []
all_y = []
for line in f:
    if cur == -1: # Skip first line
        cur += 1
        continue
    parts = line.split('\t')
    if cur < int(parts[1]):
        all_X.append(parts[2])
        all_y.append(int(parts[3]))
        cur += 1

# Split up 0-1: negative, 2(neutral) - skip, 3-4: positive
all_X2 = []
all_y2 = []
all_bad = []
all_good = []
for i in range(len(all_X)):
    if all_y[i] == 2:
        continue
    
    new_y = -1 if all_y[i] < 2 else 1
    
    all_y2.append(new_y)
    all_X2.append(all_X[i])
    
    if new_y == -1:
        all_bad.append(all_X[i])
    else:
        all_good.append(all_X[i])

Here are examples of one negative and one positive movie review from the sample we just loaded:

In [ ]:
# A bad example
print "Negative:"
print all_bad[8]
print
# A good one
print "Positive:"
print all_good[9]

## 'Training' the computer
Now we have to transform the movie review sentences into a format that the computer can understand and process efficiently.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer(min_df=5, stop_words='english')
vX = vectorizer.fit_transform(all_X2)

print "In human:", all_X2[763]
print "In computer:", vX[763]


Just like a teacher gives you problems you've never seen before on a test, we want to save some movie reviews to test the classifier with after we train it.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vX, all_y2, test_size=0.2, random_state=123)


Now it is time to train the classifier. X_train is a list of movie reviews in "computer language" and y_train is a of "good" and "bad labels". We show the classifier both (with the fit) function and it learns the patterns in the data.

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='linear', C=1)
clf.fit(X_train, y_train)

## How well did we do?
Let's see how well the classifier did: we show it movie reviews and it tells us what label it should have. The score shows what percent it got correct.  "something about train and test data"....

In [ ]:
# How well we are doing:
print "Accuracy on examples used in training", clf.score(X_train, y_train) * 100, "%"
print "Accuracy on new examples", clf.score(X_test, y_test) * 100, "%"

It may look like the computer did not do very well, but many sentences are confusing. Let's see how it does on specific examples

In [ ]:
# Let's look at some specific examples
bad = ["I have never seen a movie so dull and uninspired"]
textVX = vectorizer.transform(bad)
print "I have never seen a movie so dull and uninspired"
print "Prediction:", clf.predict(textVX)

good = ["What a masterpiece, everyone must see it"]
textVX = vectorizer.transform(good)
print "What a masterpiece, everyone must see it"
print "Prediction:", clf.predict(textVX)


In [ ]:
# User sentence
user_sentence = ""
textVX = vectorizer.transform([user_sentence])
print user_sentence
print clf.predict(textVX)

The algorithm does poorly on the confusing sentence, because it does not consider the word order:

In [ ]:
confusing = ["Although the plot was good, the execution was poor"]
textVX = vectorizer.transform(good)
print "Although the plot had potential, the execution was poor"
print clf.predict(textVX)


In [ ]:
confusing = ["Although the plot was poor, the execution was good"]
textVX = vectorizer.transform(good)
print "Although the plot was poor, the execution was good"
print clf.predict(textVX)

In [ ]:
confusing = ["plot Although the execution good poor the was was"]
textVX = vectorizer.transform(good)
print "plot Although the execution good poor the was was"
print clf.predict(textVX)

## Underlying mechanism
What words would you use to tell apart good and bad movie reviews? (Boring, exhilarating...)
Well, we can see what words computers identified as "good-movie" and "bad-movie":

In [ ]:
# What does it mean: -- try to do relative thing instead
terms = [term for _, term in sorted((i, term) for term, i in vectorizer.vocabulary_.iteritems())]
term_weights = sorted(zip(clf.coef_.toarray()[0], terms))
from pprint import pprint

# Top 100 terms:
pprint(term_weights[-10:][::-1])

In [ ]:
pprint(term_weights[:10])

In [ ]:
%matplotlib notebook

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
nterms = 25
randpoints = np.random.choice(len(terms), [1, nterms])[0]
randweights = clf.coef_.toarray()[0][randpoints]
randterms = np.array(terms)[randpoints]
plt.plot(randweights, np.arange(0, nterms), '+')
plt.yticks([])
plt.title('Term weights')
plt.xlabel('Positivity weight')
for i in xrange(nterms):
    plt.text(randweights[i] + 0.05, i - 0.15, randterms[i], size='smaller')
plt.show()

## Takeaways

- Using basic Machine Learning techniques, we were able to correctly predict the sentiment/rating of 3 out of 4 movie reviews.
- The algorithm was able to identify clearly "good" (hilarious, masterpiece) and "bad" (worst, stupid) words, and use them to guess the rating.
- The algorithm was confused by words that contained both good and bad words.
    - This happened because the algorithm looked only at individual words, not at how they were connected by words like 'although' and 'but'
    